In [1]:
import numpy as np
from igraph import *
import pandas as pd
import time

In [2]:
data = pd.read_csv('../jimi/jimi_data/graph_data.csv')

E:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (1,13,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns

Index([u'Unnamed: 0', u'orderId', u'userId', u'tag1', u'tag2', u'l_user_phone',
       u'workplace', u'createTime', u'goodsName', u'oriPri', u'num', u'price',
       u'firstPay', u'relMobile', u'province', u'city', u'region',
       u'order_ldeviceId', u'order_ip', u'order_mac', u'GPS', u'GPS_100',
       u'GPS_1000'],
      dtype='object')

## 1.使用正常的属性直接建图

### 1.1 使用UserId这个属性进行测试

In [4]:
attributes = ['userId','l_user_phone','order_ip','GPS','GPS_100','GPS_1000','order_ldeviceId'] #可以直接使用的attribute
# workplace,relMobile需要经过处理 有频率很大的数据，会生成很多无用边

In [5]:
# Initial Graph
g = Graph()
g.add_vertices(data.shape[0])

In [60]:
attribute = 'userId'
X = data[data[attribute].notnull()][attribute]
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)
#一个attribute的dic完成
for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

In [6]:
print g.ecount()

0


### 1.2 全部属性建图

In [12]:
for attribute in attributes:
    print attribute,
    X = data[data[attribute].notnull()][attribute]
    dic = {}
    for i in X.index:
        try:
            dic[X.loc[i]].append(i)
        except:
            dic[X.loc[i]] = []
            dic[X.loc[i]].append(i)
    #一个attribute的dic完成
    for L in dic.values(): #dic.values()是list的集合
        if len(L)>1:#说明存在边
            for i in xrange(len(L)):
                for j in xrange(i+1,len(L)):
                    g.add_edges([(L[i],L[j])])
    print 'graph edge #:',g.ecount()

userId graph edge #: 121139
l_user_phone graph edge #: 146274
order_ip graph edge #: 156027
GPS graph edge #: 174385
GPS_100 graph edge #: 263431
GPS_1000 graph edge #: 640941
order_ldeviceId graph edge #: 642096


## 2.Unipartite Graph 性质

In [13]:
#边数
print g.ecount()

642096


In [14]:
L = g.get_adjlist()

In [30]:
L[42982]

[1, 1, 1]

In [26]:
AdjList_new[603]

[0, 9, 43036]

In [28]:
AdjList[0]

[9, 603, 43036]

In [20]:
AdjList = g.get_adjlist()
AdjList_new = []
for AdjNode in AdjList:
    AdjList_new.append(list(set(AdjNode)))

In [32]:
edge_count = 0
for AdjNode in AdjList_new:
    edge_count += len(AdjNode)
print edge_count

980746


In [33]:
#实际共有490373条边
980746/2

490373

## 2.使用workplace,relMobile, order_mac 建图

In [7]:
# relMobile
X = data[data['relMobile'].notnull()]
X = X[X['relMobile']!='d41d8cd98f00b204e9800998ecf8427e']['relMobile']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])
print g.ecount()

2008


In [8]:
X = data[data['workplace'].notnull()]
X1 = X[X['workplace'] != '个体']
X2 = X1[X1['workplace']!='自由职业']
X3 = X2[X2['workplace']!='个体户']
X = X3[X3['workplace']!='无']['workplace']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

In [10]:
#'order_mac'
X1 = data[data['order_mac'].notnull()]
X = X1[X1['order_mac']!='未知']['order_mac']
dic = {}
for i in X.index:
    try:
        dic[X.loc[i]].append(i)
    except:
        dic[X.loc[i]] = []
        dic[X.loc[i]].append(i)

for L in dic.values(): #dic.values()是list的集合
    if len(L)>1:#说明存在边
        for i in xrange(len(L)):
            for j in xrange(i+1,len(L)):
                g.add_edges([(L[i],L[j])])

##  3. 生成unipartite graph 得到Adjlist并去重

In [35]:
import cPickle as pickle
with open('Unipartite_Graph','w') as f:
    pickle.dump(g,f)

In [36]:
with open('AdjList','w')as f:
    pickle.dump(AdjList_new,f)

In [44]:
string = ''
for i in range(len(AdjList_new)):
    line = str(i)
    for j in AdjList_new[i]:
        if j:
            line += ' '
            line += str(j)
    string+=line
    string+='\n'

In [49]:
with open('AdjList.adjlist','w')as f:
    f.write(string)

In [50]:
ls

 驱动器 D 中的卷是 Document
 卷的序列号是 0E33-1CF0

 D:\Risk_Control 的目录

2018/09/13  10:59    <DIR>          .
2018/09/13  10:59    <DIR>          ..
2018/09/12  11:29    <DIR>          .idea
2018/09/12  18:43    <DIR>          .ipynb_checkpoints
2018/09/13  10:46         9,994,609 AdjList
2018/09/13  10:59         6,636,844 AdjList.adjlist
2018/09/12  18:06               556 Basic Feature.ipynb
2018/09/12  19:07             2,274 Bipartite Graph.ipynb
2018/09/13  10:41    <DIR>          DeepWalk
2018/09/12  21:28    <DIR>          Document
2018/09/12  18:43            63,939 Homophilic Test.ipynb
2018/09/12  09:33             1,085 LICENSE
2018/09/12  09:33             1,007 README.md
2018/09/12  18:47    <DIR>          Supervised Model
2018/09/13  10:42            10,974 Unipartite Graph.ipynb
2018/09/13  10:44        17,902,984 Unipartite_Graph
               9 个文件     34,614,272 字节
               7 个目录 665,177,092,096 可用字节
